# Imports and Set Up

#

1- Import all the libraries
2- Download the Summary History - Summary Statistics: earnings suprises in past quarters, number of analysts, etc.
3- Compustat quarterly variables: firm performance, size, leverage, profitability, liquidity, growth, age
4- Industry level variables: HHI, R&D, etc.
5- Execucomp variables: CEO characteristics, overconfidence, age, tenure, gender, education

In [ ]:
import os
import wrds
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [ ]:
previously_downloaded_ratios = True
previously_downloaded_age = True
previously_downloaded_compustat_quarterly = True
previously_downloaded_compustat_annual = True
previously_downloaded_segments = True
previously_downloaded_estimates = True
previously_downloaded_execucomp = True
previously_downloaded_Loughran_McDonald = True
ind_col = 'sich4'


In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

In [ ]:

# Access environment variables for WRDS username and password
wrds_username = os.environ.get("WRDS_USERNAME")
wrds_password = os.environ.get("WRDS_PASSWORD")

# 1 - Base dataframe: transcripts

In [ ]:
transcripts = pd.read_pickle("../data/sxp1500_presentations_ceo_aggregated.pkl")

In [ ]:
transcripts.gvkey.nunique()
transcripts.columns
# check to see if there are duplicate transcriptid values per keydevid
transcripts.groupby(['keydevid'])['transcriptid'].apply(lambda x: x.nunique()>1).sum()
# number of words in transcript_text
transcripts['word_count_total'] = transcripts['transcript_text'].str.split().str.len()
transcripts = transcripts.drop_duplicates(subset=['keydevid'], keep='first')
database = transcripts[['companyid', 'keydevid', 'transcriptid', 'mostimportantdateutc', 'mostimportanttimeutc', 'gvkey', 'companyname', 'word_count_total']]
database['mostimportantdateutc_dt'] = pd.to_datetime(database['mostimportantdateutc'], format='%Y-%m-%d')
#database['year'] = database['mostimportantdateutc_dt'].dt.year
#database['month'] = database['mostimportantdateutc_dt'].dt.month
# 2 - Promises identified
promises = pd.read_csv("../data/sxp1500_presentations_ceo_aggregated_promises_expanded_cleaned_transcriptlevel_horizon_specificity.csv")
# create promise_id column, it is gvkey_transcriptid_2digitnumber (01, 02, 03, ...)

promises['promise_id'] = promises.groupby(['gvkey', 'transcriptid']).cumcount() + 1
promises['promise_id'] = promises['gvkey'].astype(str) + '_' + promises['transcriptid'].astype(str) + '_' + promises['promise_id'].apply(lambda x: f'{x:02d}')


list(promises.columns)
promises['3-promise-horizon-v2'].value_counts()
### Cleaning up the horizons column
# if it contains 'unclear' or "Unclear" in the promise, then set the promise horizon to 'unclear'
promises.loc[promises['3-promise-horizon-v2'].str.contains('unclear', case=False, na=False), '3-promise-horizon-v2'] = 'unclear'

def process_value(value):
    if pd.isna(value):
        return np.nan
    if value == 'unclear':
        return np.nan
    if not str(value).replace('-', '').replace('.', '').isdigit():
        return np.nan
    if '-' in value:
        try:
            number1, number2 = value.split('-')
            return (float(number1) + float(number2)) / 2
        except ValueError:
            return np.nan
    try:
        return float(value)
    except ValueError:
        return np.nan

promises['promise_horizon_months'] = promises['3-promise-horizon-v2'].apply(process_value)

promises['promise_horizon_months'].describe()
### Promises count
def process_promises(filtered_promises, suffix):
    # promises count
    filtered_promises[f'promises{suffix}_count'] = filtered_promises.groupby(['transcriptid'])['promise_id'].transform('nunique')

    # promises deliver date averaging
    filtered_promises[f'promises{suffix}_horizon'] = filtered_promises.groupby(['transcriptid'])[f'promise_horizon_months'].transform(lambda x: x.mean(skipna=True))

    # proportion of horizons that are nan per transcript id
    filtered_promises[f'promises{suffix}_horizon_nan'] = filtered_promises.groupby(['transcriptid'])['promise_horizon_months'].transform(lambda x: x.isna().sum()/len(x))
    
    filtered_promises[f'promises{suffix}_specificity_score'] = filtered_promises.groupby(['transcriptid'])['specificity_score'].transform(lambda x: x.mean(skipna=True))
    
    # Keeping relevant columns
    promise_columns = [column for column in filtered_promises.columns if f'promises{suffix}_' in column]
    promise_columns_keep = ['transcriptid']
    promise_columns_keep.extend(promise_columns)

    return filtered_promises[promise_columns_keep]

promises1 = promises[((promises['7-is-promise'] == 'yes') | (promises['7-is-promise'] == 'Yes')) &
                    ((promises['8-financial-guidance'] == 'no') | (promises['8-financial-guidance'] == 'No')) & 
                    ((promises['5-commitment-degree'] == 'strong-commitment') )]

promises1 = process_promises(promises1, '_1')

promises1.drop_duplicates(subset=['transcriptid'], keep='first', inplace=True)

database = pd.merge(database, promises1, on='transcriptid', how='left')
database['promises_1_count'].fillna(0, inplace=True)

# 3 - Ratios

In [ ]:

# Access environment variables for WRDS username and password
wrds_username = os.environ.get("WRDS_USERNAME")
wrds_password = os.environ.get("WRDS_PASSWORD")

In [ ]:
# Download the ratio data from WRDS
if previously_downloaded_ratios == False:
    # Connect to WRDS using the provided username and password
    db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    # download finratiofirm table
    ratios = db.get_table(library='wrdsapps_finratio_ibes', table='firm_ratio_ibes')
    # convert adate column to datetime, get year
    ratios['adate_dt'] = pd.to_datetime(ratios['adate'], format='%Y%m%d')
    ratios['year'] = ratios['adate_dt'].dt.year
    ratios = ratios[ratios['year'] >= 2005]
    ratios.to_pickle('../data/ratios.pkl')
else:
    # Load the ratio data from the pickle file
    ratios = pd.read_pickle('../data/ratios.pkl')

In [ ]:
ratios

In [ ]:
test = ratios[['gvkey', 'adate', 'qdate', 'roe']].head(10000)
ratios.gvkey.nunique()

In [ ]:
#ratio_columns = ['gvkey', 'adate', 'qdate', 'bm', 'pe_inc', 'pe_exi', 'ptb', 'npm', 'roa', 'roe', 'roce', 'cfm', 'debt_at', 'at_turn', 'rd_sale', 'adv_sale']
#ratios = ratios[ratio_columns]

In [ ]:
ratios.sort_values(by=['gvkey', 'adate', 'qdate'], inplace=True)

In [ ]:
ratios.drop_duplicates(subset=['gvkey', 'adate', 'qdate'], keep='last', inplace=True)

In [ ]:
# add a ratio_ prefix to each column name except gvkey adate and qdate
ratios.columns = ['ratio_' + column if column not in ['gvkey', 'adate', 'qdate'] else column for column in ratios.columns]


In [ ]:
# ldate is the last date between adate and qdate
ratios['ldate'] = ratios[['adate', 'qdate']].max(axis=1)


In [ ]:
# earnings volatility: standard deviation of roa over the last 12 quarters
ratios['ratio_earnings_volatility'] = ratios.groupby(['gvkey'])['ratio_roa'].transform(lambda x: x.rolling(6).std())


In [ ]:
ratios['ratio_earnings_volatility'].notna().sum()

# 5 - Compustat Quarterly

In [ ]:
previously_downloaded_compustat_quarterly = True
if previously_downloaded_compustat_quarterly == False:
    db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    # download finratiofirm table
    query = """
    SELECT gvkey, datadate, fyearq, fqtr,indfmt, consol, popsrc, datafmt, fyr, actq, atq, ibq, niq, aqcy, niy, epsfi12, oeps12, epsfxy, mkvaltq, prccq, prchq, prclq, saleq, cshoq, actq, lctq, xoprq, xrdq, intanq, txdbq, dpq, aqpq, dlttq, dlcq, seqq
    FROM comp_na_daily_all.fundq
    WHERE fyearq >= 2003
    """
    compustat = db.raw_sql(query)
    compustat.to_pickle('../data/compustat_q_2.pkl')

else:
    # Load the ratio data from the pickle file
    compustat = pd.read_pickle('../data/compustat_q_2.pkl')

In [ ]:
compustat=compustat[compustat.datafmt=='STD']
compustat=compustat[compustat.popsrc=='D']
compustat=compustat[compustat.consol=='C']
compustat=compustat[compustat.indfmt== 'INDL']

In [ ]:

compustat['datadate'] = pd.to_datetime(compustat['datadate'], format='%Y-%m-%d')
compustat['month']=compustat['datadate'].dt.month
compustat['year']=compustat['datadate'].dt.year
compustat.drop_duplicates(['gvkey','datadate'], inplace=True)
compustat['gvkey'] = compustat['gvkey'].astype(int)

In [ ]:
compustat

In [ ]:
# EPA
compustat['EPS'] = np.where(compustat['cshoq'] != 0, compustat['niq'] / compustat['cshoq'], np.nan)
# roa
compustat['roa'] = np.where(compustat['atq'] != 0, compustat['niq'] / compustat['atq'], np.nan)

# r&d share of total expense
compustat['rd_f'] = compustat['xrdq'] / compustat['xoprq'].where(compustat['xoprq'] != 0, np.nan)
compustat['rd_f'] = compustat['rd_f'].fillna(0)

# recognized intangible assets as part of total assets
compustat['intang_f'] = compustat['intanq'] / compustat['atq'].where(compustat['atq'] != 0, np.nan)
compustat['intang_f'] = compustat['intang_f'].fillna(0)

# depreciation as part of total assets
compustat['dpt_f'] = compustat['dpq'] / compustat['atq'].where(compustat['atq'] != 0, np.nan)
compustat['dpt_f'] = compustat['dpt_f'].fillna(0)

# Leverage
compustat['leverage'] = (compustat['dlttq'] + compustat['dlcq']) / compustat['seqq']
compustat.loc[compustat['seqq'] == 0, 'leverage'] = np.nan



In [ ]:
compustat = compustat.sort_values(['gvkey', 'fyearq', 'fqtr'])
compustat['mna_cash_q'] = compustat.groupby(
    ['gvkey', 'fyearq']
)['aqcy'].diff()

# First fiscal quarter
compustat['mna_cash_q'] = compustat['mna_cash_q'].fillna(compustat['aqcy'])


In [ ]:
compustat['mna_cash_q_f'] = np.where(
    compustat['atq'] > 0,
    compustat['mna_cash_q'] / compustat['atq'],
    np.nan
)
compustat['mna_cash_q_f'].replace([np.inf, -np.inf], np.nan, inplace=True)

compustat.loc[compustat['mna_cash_q_f'] < 0, 'mna_cash_q_f'] = 0


In [ ]:
(compustat['mna_cash_q_f'] < 0).sum()        # should be 0 or very close
compustat['mna_cash_q_f'].quantile(0.99)    # should be < ~0.3


In [ ]:

# 2.1: Cash flow (CF) = ibq + dpq
compustat['cf'] = compustat['ibq'] + compustat['dpq']


# 2.2: Market Value of Equity (MVE) = cshoq * prccq
compustat['mve'] = compustat['cshoq'] * compustat['prccq']

# 2.3: Book Debt 
#      A common approximation is atq - seqq - txdbq.
compustat['book_debt'] = compustat['atq'] - compustat['seqq']
compustat['book_debt'] = compustat['book_debt'] - compustat['txdbq'].fillna(0)

# 2.4: Tobin's Q = (MVE + Book Debt) / atq
compustat['tobin_q'] = (compustat['mve'] + compustat['book_debt']) / compustat['atq']

In [ ]:
# earnings_volatility
compustat.sort_values(['gvkey', 'datadate'], inplace=True)
compustat['earnings_volatility'] = compustat.groupby(['gvkey'])['roa'].transform(lambda x: x.rolling(6).std())

In [ ]:
from scipy.stats.mstats import winsorize

# Winsorize the variables at the 1% level
compustat['EPS'] = winsorize(compustat['EPS'], limits=[0.025, 0.025])
compustat['roa'] = winsorize(compustat['roa'], limits=[0.025, 0.025])
compustat['rd_f'] = winsorize(compustat['rd_f'], limits=[0.025, 0.025])
compustat['intang_f'] = winsorize(compustat['intang_f'], limits=[0.025, 0.025])
compustat['dpt_f'] = winsorize(compustat['dpt_f'], limits=[0.025, 0.025])
compustat['mna_cash_q_f'] = winsorize(compustat['mna_cash_q_f'], limits=[0.025, 0.025])
compustat['leverage'] = winsorize(compustat['leverage'], limits=[0.025, 0.025])
compustat['earnings_volatility'] = winsorize(compustat['earnings_volatility'], limits=[0.025, 0.025])
compustat['tobin_q'] = winsorize(compustat['tobin_q'], limits=[0.025, 0.025])


# Replace NaN values with 0
compustat = compustat.fillna(0)

### Merge

In [ ]:

database_temp = database[['transcriptid', 'gvkey', 'mostimportantdateutc']].merge(compustat, how='left', on='gvkey')
database_temp['mostimportantdateutc_dt'] = pd.to_datetime(database_temp['mostimportantdateutc'], format='%Y-%m-%d')
database_temp['qdate_diff'] = database_temp['mostimportantdateutc_dt'] - database_temp['datadate']
database_temp['qdate_diff'] = database_temp['qdate_diff'].dt.days
database_temp = database_temp[(database_temp['qdate_diff'] >= 0) & (database_temp['qdate_diff'] <= 75)]
database_temp.drop_duplicates(subset=['transcriptid'], inplace=True)
database_temp.drop(columns=['mostimportantdateutc_dt', 'qdate_diff','year', 'month'], inplace=True)


In [ ]:
database_temp.drop_duplicates(subset= ['transcriptid'],inplace=True)
database_temp.drop(columns=['gvkey', 'mostimportantdateutc'], inplace=True)

In [ ]:
database = database.merge(database_temp, how='left', on='transcriptid')

In [ ]:
database.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)

# 6 - Compustat annual - industry codes

In [ ]:
previously_downloaded_compustat_annual = True

if previously_downloaded_compustat_annual == False:
    db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    # download finratiofirm table
    query = """
    SELECT gvkey, datadate, datafmt, popsrc, consol, indfmt, cusip, cik,  sich,  naicsh, sale, ppent, emp, xrd, xad
    FROM comp_na_daily_all.funda
    WHERE fyear >= 2000
    """
    compustata = db.raw_sql(query)
    compustata.to_pickle('../data/compustat_a.pkl')

else:
    # Load the ratio data from the pickle file
    compustata = pd.read_pickle('../data/compustat_a.pkl')

In [ ]:
compustata=compustata[compustata.datafmt=='STD']
compustata=compustata[compustata.popsrc=='D']
compustata=compustata[compustata.consol=='C']
compustata=compustata[compustata.indfmt== 'INDL']

compustata.drop_duplicates(['gvkey','datadate'], inplace=True)


In [ ]:
compustata['datadate'] = pd.to_datetime(compustata['datadate'], format='%Y-%m-%d')

compustata['month']=compustata['datadate'].dt.month
compustata['year']=compustata['datadate'].dt.year


In [ ]:
compustata['sich'] = compustata['sich'].replace('', np.nan)
compustata = compustata.loc[compustata['sich'].notna()]
compustata.loc[:, 'sich4'] = compustata['sich'].astype(int).astype(str).str.pad(width=4, side='right', fillchar='0').astype(int)
compustata.loc[:, 'sich3'] = compustata['sich4'] // 10
compustata.loc[:, 'sich2'] = compustata['sich4'] // 100

# 7 - Create industry vars

### HHI

In [ ]:
compustata['sale_nonneg'] = compustata['sale'].clip(lower=0)

In [ ]:
total_sales = compustata.groupby(['year', ind_col])['sale_nonneg'].sum().reset_index(name=f'total_sales_{ind_col}')
compustata = compustata.merge(total_sales, on=['year', ind_col])
compustata[f'market_share_{ind_col}'] = compustata['sale_nonneg'] / compustata[f'total_sales_{ind_col}']

In [ ]:
compustata[f'squared_market_share_{ind_col}'] = compustata[f'market_share_{ind_col}'] ** 2
hhi = compustata.groupby(['year', ind_col])[f'squared_market_share_{ind_col}'].sum().reset_index(name=f'hhi_{ind_col}')
compustata = compustata.merge(hhi, on=['year', ind_col])

In [ ]:
compustata[f'hhi_{ind_col}'].describe()

In [ ]:
## Reverse HHI reversed value = (Max + Min) - original value
compustata[f'hhi_{ind_col}_reverse_ind'] = (compustata[f'hhi_{ind_col}'].max() + compustata[f'hhi_{ind_col}'].min()) - compustata[f'hhi_{ind_col}']


In [ ]:
compustata[f'hhi_{ind_col}_reverse_ind'].notna().sum()

### R&D intensity

In [ ]:
### R&D intensity
compustata['rd_intensity'] = compustata['xrd'] / compustata['sale']
compustata['rd_intensity'] = compustata['rd_intensity'].fillna(0)

# industry average per year
compustata['rd_intensity_ind'] = compustata.groupby(['year', ind_col])['rd_intensity'].transform(lambda x: x.mean(skipna=True))

# 8 - compustat segments

In [ ]:

if previously_downloaded_segments == False:

    wrds_username = os.getenv('WRDS_USERNAME')
    wrds_password = os.getenv('WRDS_PASSWORD')
    db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)

    # replace with wrdssec.forms
    query = f"""
        SELECT gvkey, srcdate, datadate, sid, sales, stype, sics1
        FROM comp_segments_hist_daily.wrds_segmerged
    """
    
    segments = db.raw_sql(query)
    segments.to_csv('../data/compustat_segments.csv')
    db.close()
    
else:
    segments = pd.read_csv('../data/compustat_segments.csv')

In [ ]:
segments.stype.value_counts()

## Number of segments for each firm

In [ ]:
segments.drop_duplicates(['gvkey', 'datadate','sid'], inplace=True)
segments = segments[['gvkey', 'datadate', 'sid', 'sales', 'stype']]
segments['gvkey'] = segments['gvkey'].astype(int)

In [ ]:
segments_n = segments[(segments['stype']=='BUSSEG') | (segments['stype']=='OPSEG')].groupby(['gvkey', 'datadate'])['sid'].nunique().reset_index()
segments_n = segments_n.rename(columns={'sid': 'n_segments'})

segments_n['year'] = segments_n['datadate'].astype(str).str.slice(0,4).astype(int)
segments_n = segments_n[['gvkey', 'year', 'n_segments']]

compustata['gvkey'] = compustata['gvkey'].astype(int)
compustata = compustata.merge(segments_n, on=['gvkey', 'year'], how='left',
                                  suffixes=('_df1', ''))
compustata['n_segments'].fillna(1,inplace=True)

In [ ]:
# describe compustata['n_segments']
print(compustata['n_segments'].describe())

## Strategy Uniqueness using Litov et al. (2012) method

In [ ]:
seg_ind = 'sics1'

segments = pd.read_csv('../data/compustat_segments.csv')
segments['year'] = segments['datadate'].str.slice(0,4).astype(int)
segments.drop_duplicates(['gvkey', 'datadate','sid'], inplace=True)
segments = segments[(segments['stype'] == 'BUSSEG') | (segments['stype'] == 'OPSEG')]
segments = segments[['gvkey', 'year', 'sales', seg_ind ]]
segments = segments[segments['sales'] > 0]
segments = segments[segments[seg_ind].notnull()]
segments = segments[segments[seg_ind] != '']
segments = segments[segments[seg_ind] != 0]
segments[seg_ind] = segments[seg_ind].astype(int)
segments['year'] = segments['year'].astype(int)
segments['gvkey'] = segments['gvkey'].astype(int)
segments = segments.rename(columns={'gvkey': 'GVKEY'})
segments = segments.rename(columns={seg_ind: 'segment_sic'})
segments = segments.rename(columns={'sales': 'segment_sale'})
segments = segments.groupby(['GVKEY', 'year', 'segment_sic'])['segment_sale'].sum().reset_index(name='segment_sale')
segments['segment_sic'] = segments['segment_sic'].astype(int)

In [ ]:

# Step 1
idx = segments.groupby(['GVKEY', 'year'])['segment_sale'].idxmax()
segments['primary_sic'] = segments.loc[idx, 'segment_sic']
segments['primary_sic'] = segments.groupby(['GVKEY', 'year'])['primary_sic'].transform('max')

# Step 2
total_sales = segments.groupby(['GVKEY', 'year'])['segment_sale'].transform('sum')
segments['norm_sale'] = segments['segment_sale'] / total_sales

# Step 3
firm_year_matrix = segments.pivot_table(index=['GVKEY', 'year', 'primary_sic'],
                                        columns='segment_sic',
                                        values='norm_sale').fillna(0)

# Step 4
actual_sales_matrix = segments.pivot_table(index=['GVKEY', 'year', 'primary_sic'],
                                           columns='segment_sic',
                                           values='segment_sale').fillna(0)

industry_year_sales = actual_sales_matrix.groupby(['primary_sic', 'year']).sum()

# Step 5
total_industry_sales = industry_year_sales.sum(axis=1)
norm_industry_year_sales = industry_year_sales.div(total_industry_sales, axis=0)

# Step 6
diff_matrix = firm_year_matrix.subtract(norm_industry_year_sales, axis=1)

# step 7: sum of squared differences
squared_diff_matrix = diff_matrix ** 2
sum_squared_diff = squared_diff_matrix.sum(axis=1)

uniqueness = sum_squared_diff.reset_index(name='strategy_unique')

In [ ]:
uniqueness = uniqueness.rename(columns={'GVKEY': 'gvkey'})

In [ ]:
uniqueness.drop_duplicates(['gvkey', 'year'], inplace=True)

In [ ]:
# gvkey to int
uniqueness['gvkey'] = uniqueness['gvkey'].astype(int)
compustata['gvkey'] = compustata['gvkey'].astype(int)

In [ ]:
compustata = compustata.merge(uniqueness, on=['gvkey', 'year'], how='left')

### Age

In [ ]:
# Download the ratio data from WRDS
if previously_downloaded_age == False:
    # Connect to WRDS using the provided username and password
    db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    # download finratiofirm table
    query = """
    SELECT gvkey, datadate 
    FROM comp_na_daily_all.funda
    """
    age = db.raw_sql(query)
    age.to_pickle('../data/compustat_age.pkl')

else:
    # Load the ratio data from the pickle file
    age = pd.read_pickle('../data/compustat_age.pkl')


In [ ]:
age.drop_duplicates(inplace=True)
age['datadate_dt'] = pd.to_datetime(age['datadate'], format='%Y-%m-%d')
age['year'] = age['datadate_dt'].dt.year
age['first_year'] = age.groupby('gvkey')['year'].transform('min')
age['firm_age'] = age['year'] - age['first_year']
age = age[['gvkey', 'datadate', 'firm_age']]
age['gvkey'] = age['gvkey'].astype(int)
age.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)


In [ ]:
age['datadate'] = pd.to_datetime(age['datadate'], format='%Y-%m-%d')


In [ ]:
compustata = compustata.merge(age, how='left', on=['gvkey', 'datadate'])

### Merge


In [ ]:
# drop year and month from compustata
compustata.drop(columns=['year', 'month'], inplace=True)
compustata['gvkey'] = compustata['gvkey'].astype(int)
compustata.rename(columns={'datadate': 'datadate_annual'}, inplace=True)
database_select = database[['gvkey', 'datadate']]
database_select_merge = pd.merge(database_select, compustata, how='outer', on=['gvkey'])
database_select_merge.sort_values(['gvkey', 'datadate_annual', 'datadate'], inplace=True)
database_select_merge['datadate_diff'] = database_select_merge['datadate'] - database_select_merge['datadate_annual']
database_select_merge['datadate_diff'] = database_select_merge['datadate_diff'].dt.days
database_select_merge = database_select_merge[(database_select_merge['datadate_diff'] >= 0) & (database_select_merge['datadate_diff'] <= 360)]
database_select_merge.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)
database = database.merge(database_select_merge, how='left', on=['gvkey', 'datadate'])
database.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)

# 9 - IBES data

In [ ]:

previously_downloaded_estimates = True
if previously_downloaded_estimates == False:

    wrds_username = os.getenv('WRDS_USERNAME')
    wrds_password = os.getenv('WRDS_PASSWORD')
    db_wrds = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)

    # replace with wrdssec.forms
    query = f"""
        SELECT *
        FROM tr_ibes.statsum_epsus
    """

    estimates = db_wrds.raw_sql(query)
    estimates.to_csv('../data/estimates_summary.csv')
    db_wrds.close()

else:
    estimates = pd.read_csv('../data/estimates_summary.csv')



In [ ]:
estimates = estimates[(estimates['fiscalp']=='QTR')]

In [ ]:
estimates = estimates.dropna(subset=['fpedats'])
estimates = estimates[estimates['fpedats'] != 'None']
estimates['year'] = estimates['fpedats'].astype(str).str.slice(0,4).astype(int)

In [ ]:
estimates = estimates.dropna(subset=['actual'])
estimates = estimates.dropna(subset=['meanest'])

In [ ]:
estimates = estimates[estimates['year']>2005]

In [ ]:
estimates['earnings_surprise'] = (estimates['actual'] - estimates['meanest'])
estimates['earnings_surprise_norm'] = (estimates['actual'] - estimates['meanest']) / estimates['meanest']

# Replace infinite values with NaN
estimates = estimates.replace([np.inf, -np.inf], np.nan)

In [ ]:
estimates['fpedats_dt'] = pd.to_datetime(estimates['fpedats'], format='%Y-%m-%d')
estimates['est_year'] = estimates['fpedats_dt'].dt.year
estimates['est_month'] = estimates['fpedats_dt'].dt.month

In [ ]:
estimates.drop(columns='year', inplace=True)

In [ ]:
test=estimates.head(10000)

In [ ]:
test=estimates.head(10000)[['ticker', 'statpers', 'fpedats', 'actdats_act', 'actual', 'meanest', 'earnings_surprise', 'earnings_surprise_norm', 'est_year', 'est_month']]

In [ ]:
estimates = estimates.sort_values(['ticker', 'fpedats', 'statpers']).drop_duplicates(['ticker', 'fpedats'], keep='last')

In [ ]:
estimates.columns

### Merge

In [ ]:
database['cusip'] = database['cusip'].str[:8]

In [ ]:
database_select = database[['cusip', 'datadate']]

In [ ]:
database_select.drop_duplicates(inplace=True)

In [ ]:
database_select = database_select.merge(estimates, how='outer', on=['cusip'])

In [ ]:
database_select['datadate_diff'] = database_select['datadate'] - database_select['fpedats_dt']
database_select['datadate_diff'] = database_select['datadate_diff'].dt.days

In [ ]:
database_select = database_select[(database_select['datadate_diff'] >= -15) & (database_select['datadate_diff'] <= 15)]

In [ ]:
database_select.shape

In [ ]:
database_select['datadate_diff'].describe()

In [ ]:
database = database.merge(database_select, how='left', on=['cusip', 'datadate'])

In [ ]:
database_select['meanest'].describe()

# 10 - Execucomp data and CEO compensation

In [ ]:
previously_downloaded_execucomp = True
if previously_downloaded_execucomp == False:
        
        wrds_username = os.getenv('WRDS_USERNAME')
        wrds_password = os.getenv('WRDS_PASSWORD')
        db_wrds = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    
        # replace with wrdssec.forms
        query = f"""
            SELECT *
            FROM  comp_execucomp.anncomp
            WHERE ceoann='CEO'
        """
        
        execucomp = db_wrds.raw_sql(query)
        execucomp['gvkey'] = execucomp['gvkey'].astype(int)

        execucomp.to_csv('../data/execucomp.csv')
        db_wrds.close()

else:
    execucomp = pd.read_csv('../data/execucomp.csv')
    

In [ ]:
execucomp.shape

### ceo duality

In [ ]:
# whether the title collumn contains chmn or chairman
execucomp['ceo_dual'] = execucomp['title'].str.contains('chmn|chairman', case=False)

In [ ]:
execucomp['ceo_dual'].value_counts()

In [ ]:
execucomp[execucomp['ceo_dual']==1][['title']]

### director

In [ ]:
execucomp['ceo_director'] = execucomp['execdir']

### options compensation

In [ ]:
execucomp['ceo_options_compensation'] = execucomp['opt_unex_exer_est_val']/execucomp['salary']
execucomp['ceo_options_compensation_log'] = np.log(execucomp['ceo_options_compensation'])

### option awards

In [ ]:
execucomp['ceo_option_awards'] = execucomp['option_awards_blk_value']/execucomp['salary']
execucomp['ceo_option_awards_log'] = np.log(execucomp['ceo_option_awards'])

### ceo gender

In [ ]:
execucomp['ceo_gender'] = execucomp['gender']

In [ ]:
execucomp['ceo_gender'].value_counts()

In [ ]:
# gender dummy
execucomp['ceo_gender_dummy'] = execucomp['ceo_gender'].apply(lambda x: 1 if x=='MALE' else 0)

### ceo age

In [ ]:
execucomp['ceo_age'] = execucomp['age']

### ceo tenure

In [ ]:
# First, make sure your DataFrame is sorted
execucomp = execucomp.sort_values(by=['gvkey', 'execid', 'year'])

# Create a group identifier for consecutive years
execucomp['year_group'] = execucomp.groupby(['gvkey', 'execid'])['year'].diff().ne(1).cumsum()

# Create the ceo_tenure variable
execucomp['ceo_tenure'] = execucomp.groupby(['gvkey', 'execid', 'year_group']).cumcount() + 1

# Drop the temporary 'year_group' column
execucomp.drop(columns=['year_group'], inplace=True)

In [ ]:
test = execucomp[['gvkey', 'execid', 'year', 'ceo_tenure']]

In [ ]:
test.head(200)

### other vars

In [ ]:
execucomp['ceo_total_compensation'] = execucomp['salary'] + execucomp['bonus'] + execucomp['othcomp']
execucomp['ceo_total_compensation_log'] = np.log(execucomp['ceo_total_compensation'])

execucomp['ceo_total_deferred_compensation'] = execucomp['defer_balance_tot'] + execucomp['defer_contrib_co_tot'] + execucomp['defer_contrib_exec_tot']
execucomp['ceo_total_deferred_compensation_log'] = np.log(execucomp['ceo_total_deferred_compensation'])

execucomp['ceo_total_shares_owned'] = execucomp['shrown_tot']
execucomp['ceo_total_shares_owned_log'] = np.log(execucomp['ceo_total_shares_owned'])



In [ ]:
execucomp['ceo_years_left'] = execucomp.groupby(['gvkey', 'execid'])['year'].transform('max') - execucomp['year']


In [ ]:
execucomp['ceo_years_left'].describe()

### Merge

In [ ]:
database['year'] = database['datadate'].dt.year

In [ ]:
execucomp_select = execucomp[['exec_fullname', 'gvkey', 'execid', 'year', 'title', 'ceo_dual', 'ceo_director', 'ceo_options_compensation', 'ceo_options_compensation_log', 'ceo_option_awards', 'ceo_option_awards_log', 'ceo_gender', 'ceo_gender_dummy', 'ceo_age', 'ceo_tenure', 'ceo_total_compensation', 'ceo_total_compensation_log', 'ceo_total_deferred_compensation', 'ceo_total_deferred_compensation_log', 'ceo_total_shares_owned', 'ceo_total_shares_owned_log', 'ceo_years_left'
]]

In [ ]:
execucomp_select.drop_duplicates(subset=['gvkey', 'year'], inplace=True)

In [ ]:
database = database.merge(execucomp_select, how='left', on=['gvkey', 'year'], suffixes=('', '_excomp'))

# 11 - Execucomp - CEO Functional Backgorund

In [ ]:
previously_downloaded_execucomp = True
if previously_downloaded_execucomp == False:
        
        wrds_username = os.getenv('WRDS_USERNAME')
        wrds_password = os.getenv('WRDS_PASSWORD')
        db_wrds = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
    
        # replace with wrdssec.forms
        query = f"""
            SELECT *
            FROM  comp_execucomp.anncomp
            WHERE ceoann='CEO'
        """
        
        execucomp = db_wrds.raw_sql(query)
        execucomp['gvkey'] = execucomp['gvkey'].astype(int)

        execucomp.to_csv('../data/execucomp.csv')
        db_wrds.close()

else:
    execucomp = pd.read_csv('../data/execucomp.csv')
    

In [ ]:
execucomp.head()

In [ ]:
boardex = pd.read_stata('../data/NA - BoardEx - Organization - Composition of Officers, Directors and Senior Managers.dta')

In [ ]:
boardex.head()

In [ ]:
# covert DateEndRole and DateStartRole to datetime - format is YYYY-MM-DD
boardex['DateEndRole'] = pd.to_datetime(boardex['DateEndRole'], format='%Y-%m-%d')
boardex['DateStartRole'] = pd.to_datetime(boardex['DateStartRole'], format='%Y-%m-%d')


In [ ]:
# RoleStartYear is the year when the role started
# if month is greater than 6, then it is the next year
boardex['RoleStartYear'] = boardex['DateStartRole'].dt.year
boardex['RoleEndYear'] = boardex['DateEndRole'].dt.year

# if month is greater than 6, then it is the next year
boardex['RoleStartYear'] = np.where(boardex['DateStartRole'].dt.month > 6, boardex['DateStartRole'].dt.year + 1, boardex['DateStartRole'].dt.year)

boardex['RoleEndYear'] = np.where(boardex['DateEndRole'].dt.month > 6, boardex['DateEndRole'].dt.year + 1, boardex['DateEndRole'].dt.year)

In [ ]:
# if roleendyera is empty, replace with 2024
boardex['RoleEndYear'] = np.where(boardex['RoleEndYear'].isna(), 2024, boardex['RoleEndYear'])



In [ ]:
boardex['RoleLength'] = boardex['RoleEndYear'] - boardex['RoleStartYear'] + 1

In [ ]:
titles_cleaned = pd.read_pickle('../data/ceo_funcational_background_titles2_ceo_role.pkl')

In [ ]:
titles_cleaned.columns

In [ ]:
titles_cleaned = pd.merge(boardex, titles_cleaned, on=['RoleName'], how='left')

In [ ]:
titles_cleaned[['RoleName', 'primary_functional_area']].sample(20)

In [ ]:
titles_cleaned['primary_functional_area'].value_counts()

In [ ]:
titles_cleaned

In [ ]:
results_df = titles_cleaned.copy()

In [ ]:
results_df.columns

In [ ]:

def pick_functional_background(group):
    """
    Given all the roles for one DirectorID, return exactly one final functional background.
    
    Rules:
      1) If there's any recognized category besides Other/Unclear,
         choose the one with the largest total RoleLength.
      2) If only Other/Unclear remains => Other/Unclear
    """
    cats = group['primary_functional_area'].unique()
    
    # 1) First check for any non-Other/Unclear categories
    cat_sums = group.groupby('primary_functional_area', dropna=False)['RoleLength'].sum()
    # ignore Other/Unclear if there's anything else
    cat_sums_meaningful = cat_sums.drop(['Other/Unclear'], errors='ignore')
    
    if not cat_sums_meaningful.empty:
        # largest sum among meaningful categories
        top_cat = cat_sums_meaningful.idxmax()
        return top_cat
    
    # 2) If only Other/Unclear is left:
    return 'Other/Unclear'

def derive_final_backgrounds(df):
    # 1) Mark CEO roles; convert Yes or No to 1 and 0
    df['is_ceo_role'] = df['is_ceo_role'].map({'Yes': 1, 'No': 0})
    # 2) Filter to DirectorIDs who were ever CEO
    ceo_ids = (
        df.groupby('DirectorID')['is_ceo_role']
          .any()
          .loc[lambda s: s].index
    )
    df_ceos = df[df['DirectorID'].isin(ceo_ids)].copy()
    
    # 3) Group by DirectorID and pick final background
    result = (
        df_ceos
        .groupby('DirectorID')
        .apply(pick_functional_background)
        .reset_index()                # no 'name=' arg here
        .rename(columns={0: 'functional_background'})
    )
    
    return result


In [ ]:
final_backgrounds = derive_final_backgrounds(results_df)


In [ ]:
final_backgrounds.sample(20)

In [ ]:
final_backgrounds['functional_background'].value_counts()

In [ ]:
boardex = boardex.merge(final_backgrounds, on='DirectorID', how='inner')

In [ ]:
boardex.head()

In [ ]:
boardex['functional_background'].value_counts()

In [ ]:
# Function to generate a list of years between RoleStartYear and RoleEndYear.
def generate_years(row):
    # Check if both start and end years are available.
    if pd.notnull(row['RoleStartYear']) and pd.notnull(row['RoleEndYear']):
        return list(range(int(row['RoleStartYear']), int(row['RoleEndYear']) + 1))
    elif pd.notnull(row['RoleEndYear']):
        # Optionally, if RoleStartYear is missing, use just the end year.
        return [int(row['RoleEndYear'])]
    else:
        return np.nan


In [ ]:

# Create the 'year' column with the list of years.
boardex['year'] = boardex.apply(generate_years, axis=1)

# Explode the DataFrame so that each year becomes its own row.
boardex_year = boardex.explode('year')

In [ ]:
boardex_year.head()

In [ ]:
import pandas as pd
import re
from thefuzz import fuzz, process



In [ ]:

# ---------------------------------------------------------
# 2) Define a cleaning function for names and companies
# ---------------------------------------------------------
def clean_text(s: str) -> str:
    """
    Lowercases the string, removes parentheses and their contents,
    removes common corporate suffixes, punctuation, and extra whitespace.
    """
    if not s:
        return ''
    
    # Lowercase
    s = s.lower()
    
    # Remove anything inside parentheses
    s = re.sub(r'\(.*?\)', '', s)
    
    # Remove common legal suffixes (extend this list to fit your needs)
    remove_list = [
        'inc', 'inc.', 'corp', 'corp.', 'co', 'co.', 'corporation',
        'ltd', 'ltd.', 'plc', 'plc.', 'llc', 'limited'
    ]
    # Use regex to remove each suffix as a standalone word
    for word in remove_list:
        s = re.sub(r'\b' + word + r'\b', '', s)
    
    # Remove punctuation (except whitespace)
    s = re.sub(r'[^\w\s]', '', s)
    
    # Remove extra whitespace
    s = re.sub(r'\s+', ' ', s)
    
    return s.strip()


In [ ]:

# ---------------------------------------------------------
# 3) Create "clean" columns in both dataframes
# ---------------------------------------------------------
boardex_year['clean_company'] = boardex_year['CompanyName'].apply(clean_text)
boardex_year['clean_name']    = boardex_year['DirectorName'].apply(clean_text)

execucomp['clean_company']    = execucomp['coname'].apply(clean_text)
execucomp['clean_name']       = execucomp['exec_fullname'].apply(clean_text)


In [ ]:
boardex_year = boardex_year[boardex_year['year'] >= 2009]
execucomp = execucomp[execucomp['year'] >= 2009]

In [ ]:
boardex_year.head()

In [ ]:

# ------------------------------------------------------------------------
# 3) Extract Unique Company Names from Each Side
# ------------------------------------------------------------------------
unique_boardex_companies = pd.DataFrame(
    boardex_year['clean_company'].unique(),
    columns=['clean_company']
).dropna()

unique_execucomp_companies = pd.DataFrame(
    execucomp['clean_company'].unique(),
    columns=['clean_company']
).dropna()


In [ ]:

# ------------------------------------------------------------------------
# 4) Fuzzy-Match Unique Company Names
#    We'll create a function that, for each BoardEx company,
#    finds the best match in Execucomp's unique list.
# ------------------------------------------------------------------------
def fuzzy_match_companies(source_df, target_df, col='clean_company', threshold=80):
    """
    For each 'clean_company' in source_df, find the best fuzzy match 
    in the 'clean_company' of target_df using thefuzz.process.extractOne.
    Returns a DataFrame with columns:
      [source_company, matched_company, match_score]
    Only returns rows with match_score >= threshold.
    """
    matched_rows = []
    target_companies = target_df[col].tolist()

    i = 0
    total = len(source_df)
    
    for src_val in source_df[col]:
        match_val, score = process.extractOne(
            src_val, 
            target_companies, 
            scorer=fuzz.token_set_ratio
        )

        if score >= threshold:
            matched_rows.append((src_val, match_val, score))
        
        print(i, 'of', total, 'done')
        i += 1
    
    out_df = pd.DataFrame(matched_rows, columns=[
        f"{col}_boardex", f"{col}_execucomp", 'company_score'
    ])
    return out_df

company_matches = fuzzy_match_companies(
    source_df=unique_execucomp_companies,
    target_df=unique_boardex_companies,
    col='clean_company',
    threshold=80
)


In [ ]:

# ------------------------------------------------------------------------
# 5) Merge the Matched Company Table Back to Original Data
#    We'll do a left merge on boardex_year, and also left merge on execucomp
#    so each row in boardex_year now has the best matched execucomp company,
#    and vice versa. 
#    Then we can join by "clean_company_execucomp" or something similar.
# ------------------------------------------------------------------------

# Merge the company matches to boardex_year
boardex_merged = pd.merge(
    boardex_year,
    company_matches,
    left_on='clean_company',
    right_on='clean_company_boardex',
    how='left'
)

# Merge the company matches to execucomp
execucomp_merged = pd.merge(
    execucomp,
    company_matches,
    left_on='clean_company',
    right_on='clean_company_execucomp',
    how='left'
)

# Now we have:
# boardex_merged:
#   - clean_company_boardex
#   - clean_company_execucomp (the matched one)
#   - company_score
#
# execucomp_merged: (similarly)
#   - clean_company_boardex
#   - clean_company_execucomp
#   - company_score


In [ ]:
# only keep the rows where company_score >= 90
boardex_merged = boardex_merged[boardex_merged['company_score'] >= 90]
execucomp_merged = execucomp_merged[execucomp_merged['company_score'] >= 90]


In [ ]:

# ------------------------------------------------------------------------
# 6) Next step: Merge boardex_merged and execucomp_merged on:
#    1) year (exact)
#    2) The matched company name from each side
#       i.e. boardex_merged['clean_company_execucomp'] 
#            vs execucomp_merged['clean_company_execucomp']
#
#    Because they share the *same* matched Execucomp company name.
# ------------------------------------------------------------------------

final_df = pd.merge(
    boardex_merged,
    execucomp_merged,
    on=['year', 'clean_company_execucomp'],  # also consider merging on 'clean_company_boardex' if you prefer
    suffixes=('_boardex','_execucomp'),
    how='inner'
)

# At this point, final_df has columns from both sides, aligned by:
# - year
# - the fuzzy matched company name in execucomp


In [ ]:

# ------------------------------------------------------------------------
# 7) If you need to fuzzy match on the *person's name*:
#    You can do it now that you have a smaller dataset 
#    (only the matched companies for the same year).
# ------------------------------------------------------------------------
final_df['name_score'] = final_df.apply(
    lambda x: fuzz.token_set_ratio(x['clean_name_boardex'], x['clean_name_execucomp']),
    axis=1
)

name_threshold = 80
final_df = final_df[final_df['name_score'] >= name_threshold]


In [ ]:
final_df.head()

In [ ]:
# select CompanyID	DirectorID DirectorName	CompanyName RoleName functional_background	year exec_fullname	gvkey	execid 
final_df_select = final_df[[
    'CompanyID',
    'DirectorID',
    'DirectorName',
    'CompanyName',
    'RoleName',
    'functional_background',
    'year',
    'exec_fullname',
    'gvkey',
    'execid'
]]



In [ ]:
# get unique execid and functional_background. visualize the distribution
final_df_select.drop_duplicates(subset=['execid', 'functional_background']).groupby('functional_background').size().plot(kind='bar')

In [ ]:
final_df_select.drop_duplicates(subset=['gvkey', 'year', 'execid'], inplace=True)

In [ ]:
database = database.merge(final_df_select[['gvkey', 'year', 'execid', 'functional_background']], how='left', on=['gvkey', 'year', 'execid'])

# 12 - Add FLS, Guidance, Sentiment

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

import pandas as pd

## FLS function

In [ ]:

import ssl
# Bypass SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

# Ensure you have the required nltk resources
nltk.download('punkt')


In [ ]:

def is_forward_looking(sentence, current_year):
    # Define the keywords and conjugations for forward-looking criteria
    keywords = [
        "will", "future", "next fiscal", "next month", "next period", "next quarter", "next year",
        "incoming fiscal", "incoming month", "incoming period", "incoming quarter", "incoming year",
        "coming fiscal", "coming month", "coming period", "coming quarter", "coming year",
        "upcoming fiscal", "upcoming month", "upcoming period", "upcoming quarter", "upcoming year",
        "subsequent fiscal", "subsequent month", "subsequent period", "subsequent quarter", "subsequent year",
        "following fiscal", "following month", "following period", "following quarter", "following year"
    ]

    excluded_keywords = ["shall", "should", "can", "could", "may", "might"]

    verbs = [
        "aim", "anticipate", "assume", "commit", "estimate", "expect",
        "forecast", "foresee", "hope", "intend", "plan", "project",
        "seek", "target"
    ]

    verb_conjugations = [
        "we ", "and ", "but ", "do not ", "company ", "corporation ", "firm ", "management ",
        "and ", "but ", "does not ", "is ", "are ", "not ", "is ", "are ", "not ",
        "normally ", "normally ", "currently ", "currently ", "also ", "also "
    ]

    # Search 1: Keyword based search
    for keyword in keywords:
        if keyword in sentence and not any(excluded in sentence for excluded in excluded_keywords):
            return True

    # Search 2: Verb conjugation based search
    for verb in verbs:
        for conj in verb_conjugations:
            if f"{conj}{verb}" in sentence:
                return True

    # Search 3: Year reference based search
    year_matches = re.findall(r'\b(20\d{2})\b', sentence)
    for year in year_matches:
        if int(year) > current_year:
            return True

    return False

def measure_forward_looking_statements(transcript, current_year):
    sentences = sent_tokenize(transcript)
    total_sentences = len(sentences)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)

    forward_looking_sentences = 0
    forward_looking_words = 0

    for sentence in sentences:
        if is_forward_looking(sentence, current_year):
            forward_looking_sentences += 1
            forward_looking_words += len(word_tokenize(sentence))

    forward_looking_sentence_ratio = forward_looking_sentences / total_sentences if total_sentences > 0 else 0
    forward_looking_word_ratio = forward_looking_words / total_words if total_words > 0 else 0

    return forward_looking_sentence_ratio, forward_looking_word_ratio


In [ ]:

# Example usage
call_transcript = """
Thank you, Brett. It was a record third quarter powered by the continued strength of Microsoft Cloud, which surpassed $35 billion in revenue, up 23%. Microsoft Copilot and Copilot stack spanning everyday productivity, business process and developer services to models, data and infrastructure are orchestrating a new era of AI transformation driving better business outcomes across every role and industry. Now I'll highlight examples walking up the stack, starting with AI infrastructure.

Azure again took share as customers use our platforms and tools to build their own AI solutions. We offer the most diverse selection of AI accelerators, including the latest from NVIDIA, AMD, as well as our own first-party silicon. Our AI innovation continues to build on our strategic partnership with OpenAI, more than 65% of the Fortune 500 now use Azure OpenAI service. We also continue to innovate and partner broadly to bring customers the best selection of frontier models in open-source models, LLMs, and SLMs with 53, which we announced earlier this week, we offer the most capable and cost-effective SLM available.

It's already being trialed by companies like CallMiner, LTIMindtree, PwC, and TCS. Our models as a service offering makes it easy for developers to use LLM and SLM without having to manage any underlying infrastructure. Hundreds of paid customers from Accenture and EY to Schneider Electric are using it to take advantage of API access to third-party models, including as of this quarter, the latest from Cohere, Meta, and Mistral. And as part of our partnership announced last week, G42 will run its AI applications and services on our cloud.

All up, the number of Azure AI customers continues to grow and average spend continues to increase. We also saw an acceleration of revenue from migrations to Azure. Azure Arc continues to help customers like DICK'S Sporting Goods and World Bank streamlined their cloud migrations. Arc now has 33,000 customers, up over 2x year over year, and we are the hyperscale platform of choice for SAP and Oracle workloads with Conduent and Medline moving their on-premise Oracle Estates to Azure and Kyndryl and L'Oreal migrating their SAP workloads to Azure.

Overall, we are seeing an acceleration in the number of large Azure deals from leaders across industries, including billion-dollar-plus, multiyear commitments announced this month from Cloud Software Group and the Coca-Cola Company. The number of $100 million-plus Azure deals increased over 80% year over year, while the number of $10 million-plus deals more than doubled. Now on to data and analytics. Our Microsoft intelligent data platform provides customers with the broadest capability, spanning databases, analytics, business intelligence, governance, and AI.

Over half of our Azure AI customers also use our data and analytics tools. Customers are building intelligent applications running on Azure, PostgreSQL, and Cosmos DB with deep integrations with Azure AI. TomTom is a great example. They've used Cosmos DB along with Azure Open AI service to build their own immersive in-car infotainment system.

We are also encouraged by our momentum with our next-generation analytics platform, Microsoft Fabric. Fabric now has over 11,000 paid customers, including leaders in every industry from ABB, EDP, Energy Transfer to Equinor, Foot Locker, ITOCHU, and Lumen, and we are seeing increased usage intensity. Fabric is seamlessly integrated with Azure AI studio meaning customers can run models against enterprise data that's consolidated in Fabric's multi-cloud data lake, OneLake. And Power BI, which is also natively integrated with Fabric provides business users with AI-powered insights.

We now have over 350,000 paid customers. Now on to developers. GitHub Copilot is bending the productivity curve for developers. We now have 1.8 million paid subscribers with growth accelerating to over 35% quarter over quarter and continues to see increased adoption from businesses in every industry, including Itau, Lufthansa Systems, Nokia, Pinterest, and Volvo cars.

Copilot is driving growth across the broader GitHub platform, too. AT&T, Citigroup, and Honeywell all increased their overall getup usage after seeing productivity and code quality increases with Copilot. All up more than 90% of the Fortune 100 are now GitHub customers and revenue accelerated over 45% year over year. Anyone can be a developer with new AI-powered features across our low-code, no-code tools, which makes it easier to build an app, automate workflow or create a Copilot using natural language.

Thirty thousand organizations across every industry have used Copilot studio to customize Copilot for Microsoft 365 or build their own, up 175% quarter over quarter. Cineplex, for example, built a Copilot for customer service agents, reducing query handling time from as much as 15 minutes to 30 seconds. All up over 330,000 organizations, including over half of Fortune 100 have used AI-powered capabilities in Power Platform, and Power Apps now has over 25 million monthly active users, up over 40% year over year. Now on to future of work.

We are seeing AI democratize expertise across the workforce. What inventory turns are to efficiency of supply chains, knowledge turns, the creation and diffusion, and knowledge are to productivity of an organization and Copilot for Microsoft 365 is helping increase knowledge turns. Thus, having a cascading effect changing work, work artifacts, and workflows, and driving better decision-making, collaboration and efficiency. This quarter, we made Copilot available to organizations of all types and sizes from enterprises to small businesses, nearly 60% of the Fortune 500 now use Copilot and we have seen accelerated adoption across industries and geographies with companies like Amgen, BP, Cognizant, Koch Industries, Moody's, Novo Nordisk, NVIDIA, and Tech Mahindra purchasing over 10,000 seats.

We're also seeing increased usage intensity from early adopters, including a nearly 50% increase in the number of Copilot-assisted interactions per user in Teams, bridging group activity with business process workflows and enterprise knowledge. And we're not stopping there. We're accelerating our innovation, adding over 150 Copilot capabilities since the start of the year. With Copilot in Dynamics 365, we are helping businesses transform every role in business function as we take share with our AI-powered apps across all categories.

This quarter, we made our Copilot for service and Copilot for sales broadly available, helping customer service agents and sellers at companies like Land O'Lakes, Northern Trust, Rockwell Automation, and Toyota Group generate role-specific insights and recommendations from across Dynamics 365 and Microsoft 365, as well as third-party platforms like Salesforce, ServiceNow, and Zendesk. And with our Copilot for finance, we are drawing context from dynamics, as well as ERP systems like SAP to reduce labor-intensive processes like collections and contract and invoice capture for companies like dentsu and IDC. ISVs are also building their own Copilot integrations. For example, new integrations between Adobe Experience Cloud and Copilot will help marketers access campaign insights in the flow of their work.

When it comes to devices, Copilot in Windows is now available on nearly 225 million Windows 10 and Windows 11 PCs, up two times quarter over quarter. With Copilot, we have an opportunity to create an entirely new category of devices, purpose built for this new generation of AI. All of our largest OEM partners have announced AI PCs in recent months. And this quarter, we introduced new surface devices, which include integrated NPUs to power on-device AI experiences like auto framing and live captions.

And there's much more to come in just a few weeks, we'll hold a special event to talk about our AI vision across Windows and devices. When it comes to Teams, we once again saw year-over-year usage growth. We're rolling out a new version, which is up to two times faster while using 50% less memory for all customers. We surpassed 1 million Teams rooms for the first time as we continue to make hybrid meetings better with new AI-powered features like automatic camera switching and speaker recognition.

And Teams Phone continues to be the market leader in cloud calling now with over 20 million PSTN users, up nearly 30% year over year. All of this innovation is driving growth across Microsoft 365 companies across the private and public sector, including Amadeus, BlackRock, Chevron, Ecolab, Kimberly Clark, all chose our premium E5 offerings this quarter for advanced security, compliance, voice, and analytics. Now on to industry and cross-industry clouds. We are also bringing AI-powered transformation to every industry.

In healthcare, DAX Copilot is being used by more than 200 healthcare organizations, including Providence, Stanford Health Care, and WellSpan Health. And in manufacturing, this week, at HANNOVER MESSE, customers like BMW, Siemens, and Volvo Penta, shared how they're using our cloud and AI solutions to transform factory operations. Now on to security. Security underpins every layer of the tech stack and it's our No.

1 priority. We launched our Secure Future Initiative last fall for this reason, bringing together every part of the company to advance cybersecurity protection and we are doubling down on this very important work, putting security about all else before all other features and investments. We are focused on making continuous progress across the six pillars of this initiative as we protect tenants and isolate production systems, protect identities and secrets, protect networks, protect engineering systems, monitor and detect threats, and accelerate responses and remediation. We remain committed to sharing our learnings, tools, and innovation with customers.

A great example is Copilot for security, which we made generally available earlier this month, bringing together LLM with domain-specific skills informed by our threat intelligence and 78 trillion daily security signals to provide security teams with actionable insights. Now let me talk about our consumer businesses, starting with LinkedIn. We continue to combine our unique data with this new generation of AI to transform the way members learn, sell, and get hired. Features like LinkedIn AI-assisted messages are seeing a 40% higher acceptance rate and accepted over 10% faster by jobseekers saving hires, time and making it easier to connect them to candidates.

Our AI-powered collaborative articles, which has reached over 12 million contributions are helping increase engagement on the platform, which reached a new record this quarter. New AI features are also helping accelerate LinkedIn premium growth with revenue up 29% year over year. We are also seeing strength across our other businesses with hiring, taking share for the seventh consecutive quarter. Now on to search advertising and news.

We once again took share across Bing and Edge as we continue to apply this new generation of AI to transform how people search and browse. Bing reached over 140 million daily active users, and we are particularly encouraged by our momentum in mobile. Our free Copilot apps on iOS and Android saw a surge in downloads after our Super Bowl ad and are among the highest-rated in this category. We also rolled out Copilot to our ad platform this quarter, helping marketers use AI to generate recommendations for product images, headlines, and descriptions.

Now on to gaming. We are committed to meeting players where they are by bringing great games to more people on more devices. We set third quarter records for game streaming hours, console usage, and monthly active devices. And last month, we added our first Activision Blizzard title Diablo 4 to our Game Pass service.

Subscribers played over 10 million hours within the first 10 days, making it one of our biggest first-party Game Pass launches ever. We were also encouraged by ongoing success of Call of Duty: Modern Warfare 3, which is attracting new gamers and retaining franchise loyalists. Finally, we are expanding our games to new platforms, bringing four of our fan-favorite titles to Nintendo Switch and Sony PlayStation for the first time. In fact, earlier this month, we had seven games among the top 25 on the PlayStation store more than any other publisher.

In closing, I'm energized about our opportunity ahead as we innovate to help people and businesses thrive in this new era. With that, let me turn it over to Amy.

"""

current_year = 2024
forward_looking_sentence_ratio, forward_looking_word_ratio = measure_forward_looking_statements(call_transcript, current_year)
print(f"Forward-Looking Sentence Ratio: {forward_looking_sentence_ratio:.4f}")
print(f"Forward-Looking Word Ratio: {forward_looking_word_ratio:.4f}")

### Prep the database

In [ ]:
import pandas as pd

In [ ]:
aggregated_transcripts =pd.read_pickle("../data/sxp1500_presentations_ceo_aggregated.pkl")

In [ ]:
aggregated_transcripts.head()

In [ ]:
# test the function on the first transcript
forward_looking_sentence_ratio, forward_looking_word_ratio = measure_forward_looking_statements(aggregated_transcripts['transcript_text'][0], aggregated_transcripts['year'][0])

In [ ]:
# are there duplicates for transcriptid?
aggregated_transcripts['transcriptid'].nunique() == aggregated_transcripts.shape[0]

In [ ]:
# apply the function to all transcripts, save in two new columns the forward looking sentence ratio and the forward looking word ratio
aggregated_transcripts[['forward_looking_sentence_ratio', 'forward_looking_word_ratio']] = aggregated_transcripts.apply(lambda x: pd.Series(measure_forward_looking_statements(x['transcript_text'], x['year'])), axis=1)

In [ ]:
# print summary statistics of the forward looking sentence ratio and the forward looking word ratio
print(aggregated_transcripts[['forward_looking_sentence_ratio', 'forward_looking_word_ratio']].describe())

In [ ]:
# save the dataframe with the new columns; only transcriptid and the two new columns
aggregated_transcripts[['transcriptid', 'forward_looking_sentence_ratio', 'forward_looking_word_ratio']].to_pickle("../data/sxp1500_presentations_ceo_aggregated_forward_looking_v12.pkl")

In [ ]:
#aggregated_transcripts = pd.read_pickle("../data/sxp1500_presentations_ceo_aggregated_forward_looking_v11.pkl")

## Guidance data

In [ ]:
guidance = pd.read_stata("../data/guidance_data_bill_20240616.dta")

In [ ]:
guidance.head()

In [ ]:
# datadate column to datetime
guidance['datadate'] = pd.to_datetime(guidance['datadate'], format='%Y%m%d')

In [ ]:
# datadate column statistics
guidance['datadate'].describe()

In [ ]:
# GVKEY column rename to gvkey
guidance.rename(columns={'GVKEY': 'gvkey'}, inplace=True)

In [ ]:
# rdq column to datetime
guidance['rdq'] = pd.to_datetime(guidance['rdq'], format='%Y%m%d')

## Merge with regressions dataset

In [ ]:
# Add the two variables merging on transcriptid
database2 = database.merge(aggregated_transcripts[['transcriptid', 'forward_looking_sentence_ratio', 'forward_looking_word_ratio']], on='transcriptid', how='left')

In [ ]:
print(database2[['forward_looking_sentence_ratio', 'forward_looking_word_ratio']].describe())

In [ ]:
# datadate format to datetime
# print a sample first
print(database2['datadate'].head())

In [ ]:
# convert to datetime
database2['datadate'] = pd.to_datetime(database2['datadate'])

In [ ]:
# describe datadate
print(database2['datadate'].describe())

In [ ]:
# mostimportantdateutc format to datetime
database2['mostimportantdateutc'] = pd.to_datetime(database2['mostimportantdateutc'])

In [ ]:
# merge the guidance data with the database based on gvkey and datadate
database2 = database2.merge(guidance, on=['gvkey', 'datadate'], how='left')

In [ ]:
# save to csv
database2.to_csv("../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned_fls.csv", index=False)

## Sentiment data

In [ ]:
# load mcdonald dictionary
mcd_dic = pd.read_csv('../data/Loughran-McDonald_MasterDictionary_1993-2023.csv')

In [ ]:
mcd_dic.columns


In [ ]:
import pandas as pd
from collections import Counter
import re


In [ ]:

# Assuming `df` is your dataframe containing the earnings calls transcripts in the column 'transcript_text'
# Assuming `mcd_dic` is your dataframe containing the words categorized by 'Negative', 'Positive', etc.

# Lists of words converted to lowercase
negative_list = set(mcd_dic[mcd_dic['Negative'] > 0]['Word'].str.lower().tolist())
positive_list = set(mcd_dic[mcd_dic['Positive'] > 0]['Word'].str.lower().tolist())
uncertainty_list = set(mcd_dic[mcd_dic['Uncertainty'] > 0]['Word'].str.lower().tolist())
litigious_list = set(mcd_dic[mcd_dic['Litigious'] > 0]['Word'].str.lower().tolist())
strong_modal_list = set(mcd_dic[mcd_dic['Strong_Modal'] > 0]['Word'].str.lower().tolist())
weak_modal_list = set(mcd_dic[mcd_dic['Weak_Modal'] > 0]['Word'].str.lower().tolist())
complexity_list = set(mcd_dic[mcd_dic['Complexity'] > 0]['Word'].str.lower().tolist())

# Function to count occurrences of words from a list in a text using Counter
def count_words(text, word_set):
    words = re.findall(r'\w+', text.lower())  # Convert text to lowercase and find all words
    counter = Counter(words)
    count = sum(counter[word] for word in word_set)
    return count


In [ ]:
# Convert transcript_text column to lowercase
aggregated_transcripts['transcript_text'] = aggregated_transcripts['transcript_text'].str.lower()

In [ ]:
# Apply the function to each transcript and create new columns
aggregated_transcripts['negative_count'] = aggregated_transcripts['transcript_text'].apply(lambda x: count_words(x, negative_list))
print('negative count done')

aggregated_transcripts['positive_count'] = aggregated_transcripts['transcript_text'].apply(lambda x: count_words(x, positive_list))
print('positive count done')

aggregated_transcripts['uncertainty_count'] = aggregated_transcripts['transcript_text'].apply(lambda x: count_words(x, uncertainty_list))
print('uncertainty count done')

aggregated_transcripts['complexity_count'] = aggregated_transcripts['transcript_text'].apply(lambda x: count_words(x, complexity_list))

# Calculate total word count for each transcript
aggregated_transcripts['total_word_count'] = aggregated_transcripts['transcript_text'].apply(lambda x: len(re.findall(r'\w+', x.lower())))


In [ ]:
# calculate the ratio of each sentiment word count to the total word count
aggregated_transcripts['negative_ratio'] = aggregated_transcripts['negative_count'] / aggregated_transcripts['total_word_count']
aggregated_transcripts['positive_ratio'] = aggregated_transcripts['positive_count'] / aggregated_transcripts['total_word_count']
aggregated_transcripts['uncertainty_ratio'] = aggregated_transcripts['uncertainty_count'] / aggregated_transcripts['total_word_count']
aggregated_transcripts['complexity_ratio'] = aggregated_transcripts['complexity_count'] / aggregated_transcripts['total_word_count']

# add positve - negative / total * 100
aggregated_transcripts['sentiment_score'] = (aggregated_transcripts['positive_count'] - aggregated_transcripts['negative_count']) / aggregated_transcripts['total_word_count'] * 100

In [ ]:
# print descriptive statistics of the sentiment ratios
print(aggregated_transcripts[['negative_ratio', 'positive_ratio', 'uncertainty_ratio', 'complexity_ratio', 'sentiment_score']].describe())

In [ ]:
# add the new columns, merging on transcriptid
database2 = database2.merge(aggregated_transcripts[['transcriptid', 'negative_ratio', 'positive_ratio', 'uncertainty_ratio',  'complexity_ratio', 'sentiment_score']], on='transcriptid', how='left')

In [ ]:
database2.to_csv("../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned_fls_sentiment.csv", index=False)

# 13 - Uncertainty Data

## Risk Data

In [ ]:
# open firmquarter_2022q1.csv
uncertainty = pd.read_stata('../data/firmquarter_2022q1.dta')

In [ ]:
uncertainty.head()

In [ ]:
uncertainty.columns

In [ ]:
uncertainty = uncertainty[['gvkey', 'PRisk', 'NPRisk', 'date_earningscall']]

In [ ]:
# change date_earningscall to datetime
# it is currently like 27-Jun-2002	
uncertainty['date_earningscall_dt'] = pd.to_datetime(uncertainty['date_earningscall'], format='%d-%b-%Y')
uncertainty['gvkey'] = uncertainty['gvkey'].astype(int)



In [ ]:
uncertainty.date_earningscall_dt

In [ ]:
database2.mostimportantdateutc

In [ ]:
database2['mostimportantdateutc'] = pd.to_datetime(database2['mostimportantdateutc'])

In [ ]:
# First, sort both DataFrames by their date columns
database2 = database2.sort_values('mostimportantdateutc')
uncertainty = uncertainty.sort_values('date_earningscall_dt')

# Perform an asof merge with a 30-day tolerance
database3 = pd.merge_asof(
    database2,
    uncertainty,
    left_on='mostimportantdateutc',
    right_on='date_earningscall_dt',
    by='gvkey',
    tolerance=pd.Timedelta(days=30),
    direction='nearest'  # Options: 'backward', 'forward', or 'nearest'
)

## Economic Policy Uncertainty Data

In [ ]:
# US economic data
econ_uncertainty = pd.read_excel('../data/US_Policy_Uncertainty_Data.xlsx')

In [ ]:
econ_uncertainty.columns

In [ ]:
econ_uncertainty.columns = econ_uncertainty.columns.str.lower()

In [ ]:
# drop the last row of data in econ_uncertainty
econ_uncertainty = econ_uncertainty.iloc[:-1]


In [ ]:
# make year and month int in both dataframes; drop if an issue 
econ_uncertainty['year'] = econ_uncertainty['year'].astype(int)



In [ ]:
# create trailing average of 12 month of US economic uncertainty
econ_uncertainty['trailing_12month_uncertainty_3component'] = econ_uncertainty['three_component_index'].transform(lambda x: x.rolling(window=12).mean())
econ_uncertainty['trailing_12month_uncertainty_newsbased'] = econ_uncertainty['news_based_policy_uncert_index'].transform(lambda x: x.rolling(window=12).mean())

In [ ]:
econ_uncertainty.columns = ['year_1month_lag', 'month_1month_lag', 'three_component_index',
       'news_based_policy_uncert_index',
       'trailing_12month_uncertainty_3component',
       'trailing_12month_uncertainty_newsbased']

In [ ]:
# merge with same year but previous month in database3
database3['date_1month_lag'] = database3['mostimportantdateutc'] - pd.DateOffset(months=0)
database3['year_1month_lag'] = database3['date_1month_lag'].dt.year
database3['month_1month_lag'] = database3['date_1month_lag'].dt.month

In [ ]:
database4 = pd.merge(database3, econ_uncertainty, on=['year_1month_lag', 'month_1month_lag'], how='left')

# 14 - Slack vars

In [ ]:

###############################################################################
# STEP 1: DOWNLOAD QUARTERLY DATA (no SICH) & ANNUAL DATA (with SICH)
###############################################################################

previously_downloaded_compustat_quarterly_slack = True
if not previously_downloaded_compustat_quarterly_slack:
    db = wrds.Connection()
    
    # -------------------------------------------------------------------------
    # 1A. Query QUARTERLY data (fundq) - does NOT have SICH, but we need at, sale, che, emp, etc.
    #     We'll alias fyearq AS fyear for easy merging later.
    # -------------------------------------------------------------------------
    query_q = """
        SELECT
            gvkey,
            datadate,
            fyearq AS fyear,        -- rename so we can merge on 'fyear'
            fqtr,                   -- fiscal quarter if needed
            cusip,
            atq  AS at,             -- total assets (quarterly)
            saleq AS sale,          -- net sales (quarterly)
            cheq AS che,            -- cash & short-term investments (quarterly)
            cogsq AS cogs,          -- cost of goods sold
            revtq AS revt           -- total revenue
        FROM comp_na_daily_all.fundq
        WHERE fyearq >= 2002
          AND indfmt = 'INDL'
          AND datafmt = 'STD'
          AND popsrc = 'D'
          AND consol = 'C'
    """
    compustat_q_slack = db.raw_sql(query_q)
    
    # -------------------------------------------------------------------------
    # 1B. Query ANNUAL data (funda) - we only need gvkey, fyear, and sich for the merge
    # -------------------------------------------------------------------------
    query_a_sich = """
        SELECT
            gvkey,
            fyear,
            sich,
            emp
        FROM comp_na_daily_all.funda
        WHERE fyear >= 2002
          AND indfmt = 'INDL'
          AND datafmt = 'STD'
          AND popsrc = 'D'
          AND consol = 'C'
    """
    compustat_a_sich = db.raw_sql(query_a_sich)
    
    # Save both
    compustat_q_slack.to_pickle('compustat_q_slack.pkl')
    compustat_a_sich.to_pickle('compustat_a_sich.pkl')
    
    db.close()
else:
    compustat_q_slack = pd.read_pickle('compustat_q_slack.pkl')
    compustat_a_sich  = pd.read_pickle('compustat_a_sich.pkl')


In [ ]:

###############################################################################
# STEP 2: MERGE ANNUAL SICH INFO ONTO QUARTERLY DATA
#         We will match on (gvkey, fyear).
###############################################################################

# Just keep the columns needed for merging: gvkey, fyear, sich
compustat_a_sich = compustat_a_sich[['gvkey','fyear','sich', 'emp']].drop_duplicates()

# Merge left: keep all quarterly obs, bring in 'sich' from annual
compustat_q_slack = pd.merge(
    compustat_q_slack,
    compustat_a_sich,
    how='left',
    on=['gvkey','fyear']    # merges annual fyear with quarterly fyear
)


In [ ]:

###############################################################################
# STEP 3: BASIC CLEANING
###############################################################################

# Convert numeric columns if needed
numeric_cols = ['at','sale','che','emp','cogs','revt']
for col in numeric_cols:
    compustat_q_slack[col] = pd.to_numeric(compustat_q_slack[col], errors='coerce')

# Drop rows with missing or zero in key fields used for slack
compustat_q_slack.dropna(subset=['at','sale','che','emp'], inplace=True)
compustat_q_slack = compustat_q_slack[compustat_q_slack['at'] > 0]
compustat_q_slack = compustat_q_slack[compustat_q_slack['sale'] > 0]

# If you want a 'year' column (already have fyear from the quarterly dataset),
# just rename or confirm it:
compustat_q_slack['year'] = compustat_q_slack['fyear']

###############################################################################
# STEP 4: CREATE PERFORMANCE MEASURE
#         E.g., Vanacker et al. (2017) measure: (revt - cogs) / at
###############################################################################

compustat_q_slack['gross_profit'] = compustat_q_slack['revt'] - compustat_q_slack['cogs']
compustat_q_slack['perf_gpa'] = compustat_q_slack['gross_profit'] / compustat_q_slack['at']

###############################################################################
# STEP 5: CONSTRUCT FINANCIAL SLACK & HR SLACK
#         - FIN_SLACK = (che / at) minus industry-year mean
#         - HR_SLACK  = (emp / sale) minus industry-year mean
###############################################################################

# (a) raw ratios
compustat_q_slack['fin_slack_raw'] = compustat_q_slack['che'] / compustat_q_slack['at']

# (b) ensure sich is string for grouping
compustat_q_slack['sich'] = compustat_q_slack['sich'].fillna('0000').astype(str)

# (c) group by year & sich
group_cols = ['year','sich']

compustat_q_slack['fin_slack_mean'] = compustat_q_slack.groupby(group_cols)['fin_slack_raw'].transform('mean')

# (d) adjusted slack = raw - group mean
compustat_q_slack['fin_slack_ind_adjusted'] = compustat_q_slack['fin_slack_raw'] - compustat_q_slack['fin_slack_mean']

###############################################################################
# STEP 6: (OPTIONAL) WINSORIZE EXTREME VALUES
###############################################################################

for col in ['fin_slack_ind_adjusted','fin_slack_raw','perf_gpa']:
    lower_q = compustat_q_slack[col].quantile(0.005)
    upper_q = compustat_q_slack[col].quantile(0.995)
    compustat_q_slack[col] = np.clip(compustat_q_slack[col], lower_q, upper_q)

###############################################################################
# STEP 7: DONE! EXAMPLE PREVIEW
###############################################################################



In [ ]:
compustat_q_slack = compustat_q_slack[['gvkey', 'datadate', 'fin_slack_ind_adjusted','fin_slack_raw','perf_gpa']]

In [ ]:
compustat_q_slack['datadate'] = pd.to_datetime(compustat_q_slack['datadate'])
compustat_q_slack['gvkey'] = compustat_q_slack['gvkey'].astype(int)
compustat_q_slack.drop_duplicates(subset=['gvkey', 'datadate'], inplace=True)


In [ ]:
database5 = pd.merge(database4, compustat_q_slack, on=['gvkey', 'datadate'], how='left')

# 15 - Cleaning up Saving

In [ ]:
final_db = database5.copy()

In [ ]:
# replace database['ceo_years_left']  with nan if 0
final_db['ceo_years_left'] = final_db['ceo_years_left'].replace(0, np.nan)

In [ ]:
# winsorize earnings_surprise var in 1 percentile
from scipy.stats.mstats import winsorize

final_db['earnings_surprise_wins'] = winsorize(final_db['earnings_surprise'], limits=[0.025, 0.025])

In [ ]:
# Remove rows with missing or infinite values in 'promises_count' or 'ratio_roa'

final_db['ceo_options_compensation_log'] = np.log(final_db['ceo_options_compensation']+1)
final_db['ceo_dual_dummy'] = np.where(final_db['ceo_dual']==1, 1, 0) 
final_db['numest'].fillna(0, inplace=True)
final_db['word_count_total_log'] = np.log(final_db['word_count_total']+1)
final_db['ceo_last_3_years'] = np.where(final_db['ceo_years_left']<=3, 1, 0)
final_db['ceo_last_2_years'] = np.where(final_db['ceo_years_left']<=2, 1, 0)
final_db['ceo_last_1_years'] = np.where(final_db['ceo_years_left']<=1, 1, 0)

final_db['at_log'] = np.log(final_db['atq']+1)
final_db.sort_values(by=['gvkey','mostimportantdateutc'], inplace=True)



final_db['constant'] = 1

In [ ]:
# save to stata
# first make sure that the variables are in the right format, no infitie values, etc.
final_db.replace([np.inf, -np.inf], np.nan).to_csv('../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned.csv', index=False)

In [ ]:
final_db.drop_duplicates(subset=['execid'])['ceo_gender'].value_counts()

# Adding abnormal returns

In [ ]:
db = pd.read_csv('../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned.csv')

In [ ]:
# Create a text file with cusip and date
with open('../data/cusip_dates.txt', 'w') as f:
    for _, row in db[['cusip', 'mostimportantdateutc']].iterrows():
        f.write(f"{row['cusip']} {row['mostimportantdateutc']}\n")

In [ ]:
# Load abnormal returns data
abnormal_returns_7_edate = pd.read_csv('../data/abnormal_returns_7_edate.csv')
abnormal_returns_3_edate = pd.read_csv('../data/abnormal_returns_3_edate.csv')
abnormal_returns_7 = pd.read_csv('../data/abnormal_returns_7.csv')
abnormal_returns_3 = pd.read_csv('../data/abnormal_returns_3.csv')




In [ ]:
abnormal_returns_3

In [ ]:
abnormal_returns_3_edate

In [ ]:
# Convert cusip to lowercase in abnormal returns dataframes
abnormal_returns_3_edate['cusip'] = abnormal_returns_3_edate['cusip'].str.lower()
abnormal_returns_7_edate['cusip'] = abnormal_returns_7_edate['cusip'].str.lower()
abnormal_returns_3['cusip'] = abnormal_returns_3['cusip'].str.lower()
abnormal_returns_7['cusip'] = abnormal_returns_7['cusip'].str.lower()

# Convert cusip to lowercase in the database
db['cusip'] = db['cusip'].str.lower()

# Convert date columns to datetime for proper merging
abnormal_returns_3_edate['evtdate'] = pd.to_datetime(abnormal_returns_3_edate['evtdate'])
abnormal_returns_7_edate['evtdate'] = pd.to_datetime(abnormal_returns_7_edate['evtdate'])
abnormal_returns_3['evtdate'] = pd.to_datetime(abnormal_returns_3['evtdate'])
abnormal_returns_7['evtdate'] = pd.to_datetime(abnormal_returns_7['evtdate'])
db['mostimportantdateutc'] = pd.to_datetime(db['mostimportantdateutc'])

# Merge all abnormal returns with the database
db_with_returns_3_edate = db.merge(abnormal_returns_3_edate, 
                             left_on=['cusip', 'mostimportantdateutc'],
                             right_on=['cusip', 'evtdate'],
                             how='left')

db_with_returns_7_edate = db.merge(abnormal_returns_7_edate,
                             left_on=['cusip', 'mostimportantdateutc'],
                             right_on=['cusip', 'evtdate'],
                             how='left')

db_with_returns_3 = db.merge(abnormal_returns_3,
                             left_on=['cusip', 'mostimportantdateutc'],
                             right_on=['cusip', 'evtdate'],
                             how='left')

db_with_returns_7 = db.merge(abnormal_returns_7,
                             left_on=['cusip', 'mostimportantdateutc'],
                             right_on=['cusip', 'evtdate'],
                             how='left')

# Merge all into one database
db_merged = db.copy()

# Add 3-day event date returns
db_merged = db_merged.merge(abnormal_returns_3_edate[['cusip', 'evtdate', 'cret', 'car', 'bhar']], 
                           left_on=['cusip', 'mostimportantdateutc'],
                           right_on=['cusip', 'evtdate'],
                           how='left',
                           suffixes=('', '_3_edate'))

# Add 7-day event date returns
db_merged = db_merged.merge(abnormal_returns_7_edate[['cusip', 'evtdate', 'cret', 'car', 'bhar']], 
                           left_on=['cusip', 'mostimportantdateutc'],
                           right_on=['cusip', 'evtdate'],
                           how='left',
                           suffixes=('', '_7_edate'))

# Add 3-day returns
db_merged = db_merged.merge(abnormal_returns_3[['cusip', 'evtdate', 'ret', 'abret']], 
                           left_on=['cusip', 'mostimportantdateutc'],
                           right_on=['cusip', 'evtdate'],
                           how='left',
                           suffixes=('', '_3'))

# Add 7-day returns
db_merged = db_merged.merge(abnormal_returns_7[['cusip', 'evtdate', 'ret', 'abret']], 
                           left_on=['cusip', 'mostimportantdateutc'],
                           right_on=['cusip', 'evtdate'],
                           how='left',
                           suffixes=('', '_7'))

# Clean up duplicate columns
db_merged.drop(columns=['evtdate', 'evtdate_3_edate', 'evtdate_7_edate', 'evtdate_3', 'evtdate_7'], 
               errors='ignore', inplace=True)


# Adding CEO broken promises

In [ ]:
db = pd.read_csv('../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned.csv')

In [ ]:
# %% [markdown]
# # Load and prepare data (unchanged)

# %%
import pandas as pd
promises = pd.read_csv("../data/sxp1500_presentations_ceo_aggregated_promises_expanded_cleaned_transcriptlevel_horizon_specificity.csv")
# create promise_id column, it is gvkey_transcriptid_2digitnumber (01, 02, 03, ...)
promises['promise_id'] = promises.groupby(['gvkey', 'transcriptid']).cumcount() + 1
promises['promise_id'] = promises['gvkey'].astype(str) + '_' + promises['transcriptid'].astype(str) + '_' + promises['promise_id'].apply(lambda x: f'{x:02d}')
promises_select = promises[['gvkey', 'mostimportantdateutc', 'transcriptid', 'companyname', 'exec_fullname', 'execid', 'promise_id','1-promise-verbatim' ,'2-promise-explain' ,'3-promise-horizon-v2', 'specificity_score']].sort_values(by=['gvkey', 'mostimportantdateutc',])

labels = pd.read_csv("promises_with_keywords_v5_labels.csv")
def revert_promise_id(promise_id):
    parts = promise_id.split('_')
    fixed_parts = []
    for part in parts:
        if part.endswith('.0'):
            part = str(int(float(part)))
        fixed_parts.append(part)
    return '_'.join(fixed_parts)

labels['promise_id'] = labels['promise_id'].apply(revert_promise_id)

labels = labels[['promise_id', 'primary_keyword']]

# merge promises and labels on promise_id
promises_select = pd.merge(labels, promises_select, on=['promise_id'], how='left')

# %%
promises_select.head(5)


In [ ]:
promises_select_10percent_results = pd.read_csv("promises_select_10percent_results.csv")


In [ ]:
promises_select_10percent_results_batch2 = pd.read_csv("promises_select_10percent_batch2_results.csv")


In [ ]:
promises_select_10percent_results = pd.concat([promises_select_10percent_results, promises_select_10percent_results_batch2])

In [ ]:
# merge with promises_select_10percent on promise_id
promises_select_10percent_results_merged = pd.merge(promises_select_10percent_results, promises_select, on=['promise_id'], how='left')

# %%

promises_select_10percent_results_merged.head()


In [ ]:
promises_select_10percent_results_merged['mostimportantdateutc'] = pd.to_datetime(
    promises_select_10percent_results_merged['mostimportantdateutc'], errors='coerce'
)


In [ ]:
promises_select_10percent_results_merged['mostimportantdate_year'] = promises_select_10percent_results_merged['mostimportantdateutc'].dt.year
promises_select_10percent_results_merged['mostimportantdate_quarter'] = promises_select_10percent_results_merged['mostimportantdateutc'].dt.quarter
promises_select_10percent_results_merged['mostimportantdate_month'] = promises_select_10percent_results_merged['mostimportantdateutc'].dt.month

In [ ]:
# show the first 5 rows of the dataframe the date columns
promises_select_10percent_results_merged[['mostimportantdateutc', 'mostimportantdate_year', 'mostimportantdate_quarter', 'mostimportantdate_month']].head()


In [ ]:
# 
df = promises_select_10percent_results_merged.copy()

df['mostimportantdate'] = pd.to_datetime(df['mostimportantdateutc'], errors='coerce')
df['quarter'] = df['mostimportantdate'].dt.to_period('Q')
df['is_broken'] = df['status_code'].isin(['DELAYED', 'NOT_DELIVERED']).astype(int)

# per exec-quarter: broken count and total promises
by_qtr = (df.groupby(['execid', 'quarter'])
            .agg(broken_count=('is_broken', 'sum'),
                 promises_in_quarter=('execid', 'size'))
            .reset_index())

# cumulative *prior* quarters
by_qtr = by_qtr.sort_values(['execid', 'quarter'])
by_qtr['no_broken_promises'] = (
    by_qtr.groupby('execid')['broken_count'].cumsum().shift(1, fill_value=0).astype(int)
)
by_qtr['no_promises_prior'] = (
    by_qtr.groupby('execid')['promises_in_quarter'].cumsum().shift(1, fill_value=0).astype(int)
)

# --- Rolling windows over prior calendar quarters (8, 12, 20) ---
roll_list = []
for eid, g in by_qtr.groupby('execid', sort=False):
    q_min, q_max = g['quarter'].min(), g['quarter'].max()
    idx = pd.period_range(q_min, q_max, freq='Q', name='quarter')

    tmp = (g.set_index('quarter')[['broken_count', 'promises_in_quarter']]
             .reindex(idx, fill_value=0))

    for w in (8, 12, 20):
        tmp[f'no_broken_promises_roll{w}'] = (
            tmp['broken_count'].rolling(window=w, min_periods=1).sum()
            .shift(1, fill_value=0)
        ).astype(int)
        tmp[f'no_promises_prior_roll{w}'] = (
            tmp['promises_in_quarter'].rolling(window=w, min_periods=1).sum()
            .shift(1, fill_value=0)
        ).astype(int)

    tmp['execid'] = eid
    roll_list.append(tmp.reset_index()[['execid', 'quarter', 
        'no_broken_promises_roll8', 'no_broken_promises_roll12', 'no_broken_promises_roll20',
        'no_promises_prior_roll8', 'no_promises_prior_roll12', 'no_promises_prior_roll20']])

roll_df = pd.concat(roll_list, ignore_index=True)

# merge rolling features back to the compact per-quarter table
result = (by_qtr.merge(roll_df, on=['execid', 'quarter'], how='left'))

# Now merge back transcriptid and mostimportantdateutc from the original df
# For each execid/quarter, get the unique transcriptid and mostimportantdateutc values
meta = (df[['execid', 'quarter', 'transcriptid', 'mostimportantdateutc']]
           .drop_duplicates(subset=['execid', 'quarter', 'transcriptid', 'mostimportantdateutc']))

# If there are multiple transcriptids per execid/quarter, this will keep all combinations.
# If you want just one transcriptid per execid/quarter, you can do .drop_duplicates(['execid', 'quarter'])
# Here, let's keep all combinations for completeness.

result = result.merge(meta, on=['execid', 'quarter'], how='left')


In [ ]:

# Reorder columns to put transcriptid and mostimportantdateutc up front if you want
result = result[['transcriptid', 
                 'no_broken_promises', 'no_promises_prior', 'promises_in_quarter',
                 'no_broken_promises_roll8', 'no_broken_promises_roll12', 'no_broken_promises_roll20',
                 'no_promises_prior_roll8', 'no_promises_prior_roll12', 'no_promises_prior_roll20']]

# result is your final table


In [ ]:
# merge with db on transcriptid
db_merged = pd.merge(db, result, on=['transcriptid'], how='inner')



In [ ]:
db_merged.to_csv('../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned_with_promises_brokenpromises.csv', index=False)

# Removing product lunches

In [ ]:
db = pd.read_csv('../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned.csv')

In [ ]:
transcripts = pd.read_pickle("../data/sxp1500_presentations_ceo_aggregated.pkl")
transcripts.gvkey.nunique()
transcripts.columns
# check to see if there are duplicate transcriptid values per keydevid
transcripts.groupby(['keydevid'])['transcriptid'].apply(lambda x: x.nunique()>1).sum()
# number of words in transcript_text
transcripts['word_count_total'] = transcripts['transcript_text'].str.split().str.len()
transcripts = transcripts.drop_duplicates(subset=['keydevid'], keep='first')
database = transcripts[['companyid', 'keydevid', 'transcriptid', 'mostimportantdateutc', 'mostimportanttimeutc', 'gvkey', 'companyname', 'word_count_total']]
database['mostimportantdateutc_dt'] = pd.to_datetime(database['mostimportantdateutc'], format='%Y-%m-%d')
#database['year'] = database['mostimportantdateutc_dt'].dt.year
#database['month'] = database['mostimportantdateutc_dt'].dt.month
# 2 - Promises identified
promises = pd.read_csv("../data/sxp1500_presentations_ceo_aggregated_promises_expanded_cleaned_transcriptlevel_horizon_specificity.csv")
# create promise_id column, it is gvkey_transcriptid_2digitnumber (01, 02, 03, ...)

promises['promise_id'] = promises.groupby(['gvkey', 'transcriptid']).cumcount() + 1
promises['promise_id'] = promises['gvkey'].astype(str) + '_' + promises['transcriptid'].astype(str) + '_' + promises['promise_id'].apply(lambda x: f'{x:02d}')


list(promises.columns)
promises['3-promise-horizon-v2'].value_counts()
### Cleaning up the horizons column
# if it contains 'unclear' or "Unclear" in the promise, then set the promise horizon to 'unclear'
promises.loc[promises['3-promise-horizon-v2'].str.contains('unclear', case=False, na=False), '3-promise-horizon-v2'] = 'unclear'

def process_value(value):
    if pd.isna(value):
        return np.nan
    if value == 'unclear':
        return np.nan
    if not str(value).replace('-', '').replace('.', '').isdigit():
        return np.nan
    if '-' in value:
        try:
            number1, number2 = value.split('-')
            return (float(number1) + float(number2)) / 2
        except ValueError:
            return np.nan
    try:
        return float(value)
    except ValueError:
        return np.nan

promises['promise_horizon_months'] = promises['3-promise-horizon-v2'].apply(process_value)

promises['promise_horizon_months'].describe()


In [ ]:
promises['promise_id']

In [ ]:
labels = pd.read_csv("promises_with_keywords_v5_labels.csv")

In [ ]:
labels['promise_id']

In [ ]:
labels = labels[['promise_id', 'primary_keyword']]


In [ ]:

# merge promises and labels on promise_id
promises = pd.merge(labels, promises, on=['promise_id'], how='inner')

In [ ]:
promises['primary_keyword'].value_counts()    


In [ ]:
### Promises count
def process_promises(filtered_promises, suffix):
    # promises count
    filtered_promises[f'promises{suffix}_count_nolaunch'] = filtered_promises.groupby(['transcriptid'])['promise_id'].transform('nunique')

    # promises deliver date averaging
    filtered_promises[f'promises{suffix}_horizon_nolaunch'] = filtered_promises.groupby(['transcriptid'])[f'promise_horizon_months'].transform(lambda x: x.mean(skipna=True))

    # proportion of horizons that are nan per transcript id
    filtered_promises[f'promises{suffix}_horizon_nan_nolaunch'] = filtered_promises.groupby(['transcriptid'])['promise_horizon_months'].transform(lambda x: x.isna().sum()/len(x))
    
    filtered_promises[f'promises{suffix}_specificity_score_nolaunch'] = filtered_promises.groupby(['transcriptid'])['specificity_score'].transform(lambda x: x.isna().sum()/len(x))
    
    # Keeping relevant columns
    promise_columns = [column for column in filtered_promises.columns if f'promises{suffix}_' in column]
    promise_columns_keep = ['transcriptid']
    promise_columns_keep.extend(promise_columns)

    return filtered_promises[promise_columns_keep]

promises1 = promises[
    ((promises['7-is-promise'] == 'yes') | (promises['7-is-promise'] == 'Yes')) &
    ((promises['8-financial-guidance'] == 'no') | (promises['8-financial-guidance'] == 'No')) &
    (promises['5-commitment-degree'] == 'strong-commitment') &
    (promises['primary_keyword'] != 'Launch Announcements')
]

promises1 = process_promises(promises1, '_1')

promises1.drop_duplicates(subset=['transcriptid'], keep='first', inplace=True)


In [ ]:

database = pd.merge(db, promises1, on='transcriptid', how='left')


In [ ]:
database['promises_1_count_nolaunch'].fillna(0, inplace=True)

In [ ]:
database.to_csv('../data/sxp1500_presentations_ceo_aggregated_regression_vars_cleaned_no_launch.csv', index=False)